In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import xgboost as xgb
from subprocess import check_output
%matplotlib inline
from sklearn import datasets, svm, metrics
np.random.seed(0)

In [2]:
import data_loader
train_files, train_labels, valid_files, valid_labels, test_files, test_labels, label_dict = data_loader.load_dataset('facial_expressions/images', 'facial_expressions/data/legend.csv', valid_rate=0.1)

In [3]:
print(label_dict)


{'sadness': 0, 'fear': 1, 'anger': 2, 'surprise': 3, 'contempt': 4, 'disgust': 5, 'neutral': 6, 'happiness': 7}


In [4]:
def image_to_feature_vector(image, size=(64, 64)):
	return cv2.resize(image, size).flatten()

In [5]:
tr_images = []

for t,im_path in enumerate(train_files):
  img = cv2.imread(im_path, 0)
  img2 = image_to_feature_vector(img)
  tr_images.append(img2)

tr_images = np.asarray(tr_images)

In [6]:
ts_images = []

for t,im_path in enumerate(test_files):
  img = cv2.imread(im_path, 0)
  img2 = image_to_feature_vector(img)
  ts_images.append(img2)

ts_images = np.asarray(ts_images)

In [7]:
va_images = []

for t,im_path in enumerate(valid_files):
  img = cv2.imread(im_path, 0)
  img2 = image_to_feature_vector(img)
  va_images.append(img2)

va_images = np.asarray(va_images)

In [8]:

print (ts_images.shape)

(1378, 4096)


## SVM

In [9]:
classifier = svm.SVC(gamma=0.001)
classifier.fit(tr_images,train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
VaPredsvm = classifier.predict(va_images)

In [13]:
VaAccSVM = metrics.accuracy_score(valid_labels, VaPredsvm)
print ("Validation SVM Accuracy: %.2f%%" % (VaAccSVM * 100.0))

Validation SVM Accuracy: 50.41%


In [12]:
y_predsvm = classifier.predict(ts_images)

In [14]:
AccSVM = metrics.accuracy_score(test_labels, y_predsvm)
print ("SVM Accuracy: %.2f%%" % (AccSVM * 100.0))

SVM Accuracy: 49.85%


## Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=100)  


In [16]:
regressor.fit(tr_images, train_labels) 

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [17]:
ValPredRF = regressor.predict(va_images)

In [19]:
VaAccRF = metrics.accuracy_score(valid_labels, np.rint(ValPredRF))
print ("Validation Random Forest Accuracy: %.2f%%" % (VaAccRF * 100.0))

Validation SVM Accuracy: 59.05%


In [20]:
PredRF = regressor.predict(ts_images)

In [21]:
AccRF = metrics.accuracy_score(test_labels, np.rint(PredRF))
print ("Random Forest Accuracy: %.2f%%" % (AccRF * 100.0))

Random Forest Accuracy: 59.58%
